# Exercise: Cross-Validation with Symmetric Pair-Input Data

This exercise consists of two tasks. The first task is compulsory: you will not get the right to take the exam if you fail the first task. The second task optional: you do not have to complete the second task but a successful completion will give you an extra point in the exam.

In both tasks, use the K-nearest neighbors classifier with K=1 and Euclidean distance for learning and the concordance index for evaluation. You are encouraged to re-use your own code from the previous exercises. Use the data files `pairs.data`, `features.data`, and `labels.data` that are available in Moodle. The descriptions of these files are provided in the exercise overview, which is also available in Moodle.

Follow the general exercise guidelines of the course (listed in Moodle). Particularly,

- Describe and implement your solution directly to this Jupyter notebook file.
- Remember to describe your solution in general and add detailed comments to the critical parts of your code.
- Remember to justify your design choices and discuss your results.
- Your report must be easy to follow and your code must be runnable in Jupyter notebook.

Feel free to use markdown cells and code cells as you see appropriate.

Submit the finished work to Moodle before the **deadline Monday 18th of February 2019 at 23:59**. Late submissions will be ignored.

## Cover page

*[write your name, your student number and any other general information here]*

Marita Risku

## Task 1 (compulsory)

**You must successfully complete this task in order to get the right to take the exam.**

1. Implement the modified leave-one-out cross-validation scheme that is described in the lecture notes.

2. Estimate and report the generalisation performance of the K-nearest neighbor classifier in predicting the functional similarity of proteins. Use both the unmodified and the modified leave-one-out cross-validation.

3. Discuss your results. In particular, answer the following questions:
 - Why do the two cross-validation schemes produce notably different estimates?
 - Which scheme is appropriate for estimating the generalisation performance on which types of pairs (A, B, or C) and why?

*[write your answer to task 1 here]*

In [1]:
# Libaries and functions used in this excercise:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import KNeighborsClassifier

**First I prepare the data.**

In the data there are 
* 20 selected proteins from UniProt database
* 95 instances with 41 features (not all possible pairs present in data)

The data is in three datasets:
* pairs.data: indicates which proteins the instances concern
* features.data: 25 unigram frequency features from amino acids, 16 bigram frequency features from 4-class amino acid categorisation, a pair is represented as the sum of object features
* labels.data: positive label if and only if the pair members share Gene Ontology annotation in "Molecular function" domain

In [2]:
# read in pairs.data
df_pairs=pd.read_csv('pairs.data', header = None)
print(df_pairs.head())
pairs = df_pairs.values
pairs.shape

     0    1
0   P0   P9
1   P6  P19
2   P0   P4
3   P5  P17
4  P15  P17


(95, 2)

In [3]:
# Just to check how the members of pairs are distributed:
df_pairs_cat = df_pairs.astype('category')
df_pairs_cat[0];
df_pairs_cat[1];
print('pair[0]:\n', df_pairs[0].value_counts(), '\n', 'pair[1]:\n', df_pairs[1].value_counts())

pair[0]:
 P3     9
P0     9
P2     9
P1     8
P7     8
P5     8
P9     7
P6     6
P8     5
P4     5
P11    5
P10    4
P13    3
P12    3
P15    2
P14    2
P16    1
P17    1
Name: 0, dtype: int64 
 pair[1]:
 P19    13
P17    10
P18    10
P16     7
P15     7
P11     7
P14     7
P9      6
P8      5
P12     4
P10     3
P13     3
P4      3
P6      3
P7      3
P3      2
P2      1
P5      1
Name: 1, dtype: int64


In [4]:
# read in features.data
df_X=pd.read_csv('features.data', header = None)
print(df_X.head())
print(df_X.shape)
df_X.describe()

   0   1   2   3   4   5   6   7   8   9  ...   31  32  33  34   35   36  37  \
0  94   0  16  51  54  54  74  21  43  61 ...   52  26  55  47  225  140  43   
1  21   0   6   8  12   5  19   5   6  20 ...    6   8  10   7   32   23  11   
2  88   0  14  59  58  55  76  24  53  57 ...   57  32  56  52  227  146  45   
3  52   0   5  18  17  51  47  16  50  15 ...   19  10  21  19  184  109  15   
4  39   0  15  27  23  51  64  19  65  15 ...   25  16  31  24  194  118  16   

   38   39   40  
0  30  132   92  
1   7   26   19  
2  35  135  105  
3  10  106   75  
4  15  117  106  

[5 rows x 41 columns]
(95, 41)


,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
count,95.000000,95.0,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,...,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000,95.000000
mean,49.400000,0.0,8.305263,24.084211,27.263158,36.736842,45.336842,16.557895,40.357895,23.505263,...,25.936842,14.915789,29.105263,25.252632,155.884211,93.989474,20.326316,14.547368,93.589474,67.178947
std,24.908513,0.0,4.633536,13.965507,14.521598,21.159079,24.366707,7.555068,19.856354,17.198388,...,13.806266,6.685429,13.163840,12.070686,74.228638,44.368979,11.210218,8.157712,40.606963,28.883471
min,14.000000,0.0,1.000000,3.000000,6.000000,4.000000,9.000000,4.000000,4.000000,5.000000,...,4.000000,4.000000,10.000000,5.000000,31.000000,21.000000,6.000000,3.000000,23.000000,17.000000
25%,30.000000,0.0,5.000000,15.000000,17.000000,18.000000,25.000000,10.000000,25.500000,13.000000,...,17.000000,11.000000,20.000000,17.000000,102.000000,59.000000,12.000000,9.000000,61.500000,40.500000
50%,41.000000,0.0,7.000000,21.000000,24.000000,31.000000,44.000000,17.000000,36.000000,16.000000,...,23.000000,14.000000,25.000000,23.000000,135.000000,78.000000,17.000000,12.000000,81.000000,60.000000
75%,63.500000,0.0,12.000000,27.000000,31.000000,52.000000,64.000000,22.500000,53.000000,29.000000,...,29.000000,17.000000,34.000000,29.500000,212.000000,129.500000,23.000000,16.000000,128.000000,92.000000
max,107.000000,0.0,21.000000,64.000000,72.000000,97.000000,101.000000,35.000000,99.000000,80.000000,...,68.000000,34.000000,67.000000,61.000000,348.000000,200.000000,53.000000,40.000000,193.000000,137.000000


In [5]:
# all the features does not show, thus I will print them separately:
for i in range(41):
    print('feature', i, ':', df_X.describe()[i], '\n')

feature 0 : count     95.000000
mean      49.400000
std       24.908513
min       14.000000
25%       30.000000
50%       41.000000
75%       63.500000
max      107.000000
Name: 0, dtype: float64 

feature 1 : count    95.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: 1, dtype: float64 

feature 2 : count    95.000000
mean      8.305263
std       4.633536
min       1.000000
25%       5.000000
50%       7.000000
75%      12.000000
max      21.000000
Name: 2, dtype: float64 

feature 3 : count    95.000000
mean     24.084211
std      13.965507
min       3.000000
25%      15.000000
50%      21.000000
75%      27.000000
max      64.000000
Name: 3, dtype: float64 

feature 4 : count    95.000000
mean     27.263158
std      14.521598
min       6.000000
25%      17.000000
50%      24.000000
75%      31.000000
max      72.000000
Name: 4, dtype: float64 

feature 5 : count    95.000000
mean     36.736842
std      21.159079
min       4.0

There are five features with std = 0. I will drop those features.

In [6]:
df_Xd= df_X.drop([1,13, 19, 22, 24 ], axis=1)

In [7]:
X = df_Xd.values
X.shape

(95, 36)

In [8]:
# read in labels.data
df_y=pd.read_csv('labels.data', header = None)
print(df_y.head())
y=df_y.values.reshape(95,) # columnvector -> 1d array
y.shape

   0
0  1
1  1
2  0
3  1
4  1


(95,)

**Cross-validation with symmetric pair-input data: the modified leave-one-out cross-validation scheme (case type C)**

I need modified leave-one-out cross-validation where there are no dependencies between training and test sets caused by shared members:
* the training set is filtered to remove shared objects
* the training set contains exactly those instances that do not share objects with the test instance

Thus I will use my own modified_training_set-function to get the indices of modified training set.

In [9]:
def modified_training_set_C(test_index, pairs):
    a = pairs[test_index,0] # first pair member of test_instance
    b = pairs[test_index,1] # second pair member of test_instance
    # the training set contains exactly those instances that do not share objects with the test instance
    train_index = np.argwhere((pairs[:,0] != a)& (pairs[:,0] != b) & (pairs[:,1] != a) & (pairs[:,1] != b))
    return train_index.reshape(len(train_index,)) # shape (74,1) -> (74,)) 

In [10]:
# for testing
#test_index=2
#ind=modified_training_set_C(test_index, pairs)
#pairs[ind]

**C-index**

I will use C-index as the accuracy measure (how well the model captures the relative ordering/ranking of the data points).

In [11]:
# function calculates C-index
def cIndex(y_true, y_predicted):
    n=0 # number of true[j] that are not same than true[i]
    h_sum=0 # predicted[i] and predicted[j] are in same order than true[i] and true[j]
    for i in range(len(y_true)):
        t=y_true[i]
        p=y_predicted[i]
        for j in range((i+1), len(y_true)): # following values
            nt=y_true[j]
            np=y_predicted[j]
            if (t != nt): # following true value is not the same
                n=n+1
                if (t < nt and p < np) or (t > nt and p > np): # same order
                    h_sum = h_sum+1
                elif (p==np):
                    h_sum = h_sum + 0.5
    return h_sum/n

**The generalisation performance of the K-nearest neighbor classifier in predicting the functional similarity of proteins.**

I will first use the unmodified leave-one-out cross-validation. I will use LeaveOneOut from Scikit-learn library. I will also use K-Nearest Neighbors Classification from the Scikit-learn library using Euclidean distance and uniform weights.
K-value was given to be 1.



In [12]:
loo = LeaveOneOut()
y_predicted_loo= np.zeros(len(y))
for train_index, test_index in loo.split(X):
    X_train=X[train_index]
    y_train=y[train_index] 
    X_test=X[test_index]
    y_test=y[test_index]
    knn = KNeighborsClassifier(1)
    knn.fit(X_train,y_train)
    y_predicted_loo[test_index] = knn.predict(X_test)
C_index_loo= cIndex(y, y_predicted_loo)
print('C-index with the unmodified leave-one-out cross-validation =', C_index_loo)

C-index with the unmodified leave-one-out cross-validation = 0.7617702448210922


Next, I will use the modified leave-one-out cross-validation.

In [13]:
y_predicted_Cloo= np.zeros(len(y))
for test_index in range(len(y)):
    X_test=X[test_index].reshape(1,36)
    y_test=y[test_index].reshape(1,)
    train_index = modified_training_set_C(test_index, pairs)
    X_train=X[train_index]
    y_train=y[train_index]
    knn = KNeighborsClassifier(1)
    knn.fit(X_train,y_train)
    y_predicted_Cloo[test_index] = knn.predict(X_test)
C_index_Cloo= cIndex(y, y_predicted_Cloo)
print('C-index with the modified (case C) leave-one-out cross-validation =', C_index_Cloo)

C-index with the modified (case C) leave-one-out cross-validation = 0.6313559322033898


The generalisation performance of the K-nearest neighbor classifier in predicting the functional similarity of proteins was
* with the unmodified leave-one-out cross-validation: C-index 0.76
* with the modified (case C) leave-one-out cross-validation: C-index 0.63

In pair-input data features and labels represent pairs, not single objects. Therefore there may be strong dependencies due to common pair members. In symmetric pair-input data pair members are of the same type and labels arise from a symmetric relation.

The cross-validation training and test sets must have dependencies that are similar to those between the known instances and the(future) unknown instances. There are three types of pairs with respect to training set

    A: both members are present in training set
    B: only one member is present in training set
    C: neither member is present in training set

The scheme with unmodified leave-one-out cross-validation is appropriate for estimating the generalisation performance on type A of pairs, because dependencies between cross-validation training and test sets are then similar
to those between known instances and unknown instances.

Similarly the scheme with modified leave-one-out cross-validation above is appropriate for estimating the generalisation performance on type C of pairs.

For type A of pairs the prediction is more accurate due to common pair members and thus C-index is better.

In next section I will implement appropriate scheme for type B of pairs.



## Task 2 (optional)

**Successfully completing this task will give you an extra point in the exam.**

1. Design a leave-one-out cross-validation scheme that is appropriate for estimating the generalisation performance on the type of pairs for which the two aforementioned schemes are not appropriate.

2. Explain why your cross-validation scheme is appropriate.

3. Implement your cross-validation scheme. Estimate and report the generalisation performance as in the first task.

4. Discuss your results. In particular, compare the results to those you obtained in the first task and give reasons for any similarities or differences you observe.

*[write your answer to task 2 here]*

**Cross-validation with symmetric pair-input data: the modified leave-one-out cross-validation scheme (case type B)**

Next I will design a leave-one-out cross-validation scheme that is appropriate for estimating the generalisation performance on the type B of pairs. Now at most one member is aloud to be present in training set. Then dependencies between cross-validation training and test sets are similar to those between known instances and unknown instances.


In [14]:
#  at most one member is present in training set.
def modified_training_set_B(test_index, pairs):
    a = pairs[test_index,0] # first pair member of test_instance
    b = pairs[test_index,1] # second pair member of test_instance
    # a is not present:
    train_index_a = np.argwhere((pairs[:,0] != a) & (pairs[:,1] != a))
    # b is not present:
    train_index_b = np.argwhere((pairs[:,0] != b)& (pairs[:,1] != b))
    return train_index_a.reshape(len(train_index_a,)) , train_index_b.reshape(len(train_index_b,)) 

In [15]:
y_predicted_Bloo_a= np.zeros(len(y))
y_predicted_Bloo_b= np.zeros(len(y))
knn = KNeighborsClassifier(1)
for test_index in range(len(y)):
    X_test=X[test_index].reshape(1,36)
    #y_test=y[test_index].reshape(1,)
    train_index_a, train_index_b = modified_training_set_B(test_index, pairs)
    # first member of the pair is not present in training set:
    X_train_a=X[train_index_a]
    y_train_a=y[train_index_a]
    knn.fit(X_train_a,y_train_a)
    y_predicted_Bloo_a[test_index] = knn.predict(X_test)
    # second member of the pair is not present in training set:
    X_train_b=X[train_index_b]
    y_train_b=y[train_index_b]
    knn.fit(X_train_b,y_train_b)
    y_predicted_Bloo_b[test_index] = knn.predict(X_test)
C_index_Bloo_a= cIndex(y, y_predicted_Bloo_a)
C_index_Bloo_b= cIndex(y, y_predicted_Bloo_b)
C_index_Bloo = (C_index_Bloo_a + C_index_Bloo_b) / 2
print('C-index with the modified (case B, at most one member is present in training set) leave-one-out cross-validation =', C_index_Bloo)


C-index with the modified (case B, at most one member is present in training set) leave-one-out cross-validation = 0.6965630885122411


As we could expect the value of the C-index in case B is somewhere between case A and C:
* Case A (with the unmodified leave-one-out cross-validation): C-index 0.76
* Case B (with the modified (case B) leave-one-out cross-validation: C-index 0.70
* Case C (with the modified (case C) leave-one-out cross-validation: C-index 0.63

In case A the prediction is most accurate because both members are aloud to be present in training set. In case B only one member is aloud to be present in training set and therefore the prediction is less accurate than in case A. In case C neither member is present in training set and therefore the prediction is worst and hence also the C-index is the lowest.

In [16]:
#testing: c-index calculated from one array gives the same answer than taking average of c-index calculated from two arrays

y_2= np.hstack((y,y)) # because two predictions are made per pair
y_predicted_Bloo= np.zeros(2*len(y))
knn = KNeighborsClassifier(1)
for test_index in range(len(y)):
    X_test=X[test_index].reshape(1,36)
    #y_test=y[test_index].reshape(1,)
    train_index_a, train_index_b = modified_training_set_B(test_index, pairs)
    # only first member of the pair is present in training set:
    X_train_a=X[train_index_a]
    y_train_a=y[train_index_a]
    knn.fit(X_train_a,y_train_a)
    y_predicted_Bloo[test_index] = knn.predict(X_test)
    # only second member of the pair is present in training set:
    X_train_b=X[train_index_b]
    y_train_b=y[train_index_b]
    knn.fit(X_train_b,y_train_b)
    y_predicted_Bloo[test_index + len(y)] = knn.predict(X_test)
C_index_Bloo= cIndex(y_2, y_predicted_Bloo)
print('C-index =', C_index_Bloo)


C-index = 0.696563088512241


In [17]:
# testing: What would the C-index be, if exactly one member was present in training set.
# This should lead to little bit better c-index, because there is no case C -type of pairs involved.
def modified_training_set_B_2(test_index, pairs):
    a = pairs[test_index,0] # first pair member of test_instance
    b = pairs[test_index,1] # second pair member of test_instance
    # only a is present:
    train_index_a = np.argwhere(((pairs[:,0] == a)& (pairs[:,1] != b)) |  ((pairs[:,0] != b)& (pairs[:,1] == a)))
    # only b is present:
    train_index_b = np.argwhere(((pairs[:,0] == b)& (pairs[:,1] != a)) |  ((pairs[:,0] != a)& (pairs[:,1] == b)))
    return train_index_a.reshape(len(train_index_a,)) , train_index_b.reshape(len(train_index_b,)) 

In [18]:
y_predicted_Bloo_a= np.zeros(len(y))
y_predicted_Bloo_b= np.zeros(len(y))
knn = KNeighborsClassifier(1)
for test_index in range(len(y)):
    X_test=X[test_index].reshape(1,36)
    #y_test=y[test_index].reshape(1,)
    train_index_a, train_index_b = modified_training_set_B_2(test_index, pairs)
    # only first member of the pair is present in training set:
    X_train_a=X[train_index_a]
    y_train_a=y[train_index_a]
    knn.fit(X_train_a,y_train_a)
    y_predicted_Bloo_a[test_index] = knn.predict(X_test)
    # only second member of the pair is present in training set:
    X_train_b=X[train_index_b]
    y_train_b=y[train_index_b]
    knn.fit(X_train_b,y_train_b)
    y_predicted_Bloo_b[test_index] = knn.predict(X_test)
C_index_Bloo_a= cIndex(y, y_predicted_Bloo_a)
C_index_Bloo_b= cIndex(y, y_predicted_Bloo_b)
C_index_Bloo = (C_index_Bloo_a + C_index_Bloo_b) / 2
print('C-index with the modified (exactly one member is present in training set) leave-one-out cross-validation =', C_index_Bloo)


C-index with the modified (exactly one member is present in training set) leave-one-out cross-validation = 0.708921845574388
